In [1]:
import pandas as pd
import numpy as np
import pickle
from itertools import chain
from collections import Counter
import operator

In [2]:
data_dir = '../prd_review_v2/data/'

prev_dt = '190104'
new_dt = '190117'

In [3]:
def get_df(dt):
    df_result = pd.read_pickle(data_dir+'df_result_'+dt+'.pkl')
    df_test = pd.read_pickle(data_dir+'df_test_'+dt+'.pkl')

    with open(data_dir+'word_index_'+dt+'.pkl', 'rb') as pkl_file:
        vocab_dict = pickle.load(pkl_file)            
    vocab_list = [x for x in range(len(vocab_dict)+1)]
    for k, v in vocab_dict.items():
        vocab_list[v] = k
        
    df_test = df_test.head(len(df_result)).reset_index(drop=True)
    # df_test.x = df_test.x.apply(lambda x: x.split())
    df_test = df_test.rename({'x': 'origin_tokens'}, axis=1)
    
    assert len(pd.Series(df_result.y == df_test.y).unique()) == 1
    
    df = pd.concat([df_result, df_test.drop(['y'], axis=1)], axis=1)
    df['tokens'] = df.x.apply(lambda x_list: [vocab_list[x] for x in x_list])
    
    return df

In [5]:
df_prev = get_df(prev_dt)
df_prev.sample(10)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
41,"[16, 135, 53, 10, 19, 111, 0, 0, 0, 0, 0, 0, 0...",1,"[3.6125377e-05, 2.8032675e-05, 8.846524e-06, 7...",1,135168670,배송은 엄청 빠르진 않지만 제품은 마음에 듭니다,"[배송, 엄청, 빠르, 않, 제품, 마음]","[배송, 엄청, 빠르, 않, 제품, 마음, 0, 0, 0, 0, 0, 0, 0, 0..."
28906,"[2056, 555, 60, 49, 555, 226, 398, 15, 4, 201,...",1,"[4.3445154e-05, 5.4022647e-05, 8.706877e-05, 0...",0,135084135,온수매트에 딱맞네요 매트커버는 지퍼로 되어있어 세탁하기 쉬운데요 온수 호수쪽에 커버...,"[온수, 매트, 딱, 맞, 매트, 커버, 지퍼, 되, 있, 세탁, 쉽, 온수, 호수...","[온수, 매트, 딱, 맞, 매트, 커버, 지퍼, 되, 있, 세탁, 쉽, 온수, 호수..."
1745,"[1, 335, 855, 206, 14, 11, 0, 0, 0, 0, 0, 0, 0...",1,"[0.00016731136, 0.00024816176, 0.00015613185, ...",1,135307633,좋아요 신랑이랑 회사에서 같이 쓰려고 구매래요,"[좋, 신랑, 회사, 같이, 쓸, 구매]","[좋, 신랑, 회사, 같이, 쓸, 구매, 0, 0, 0, 0, 0, 0, 0, 0,..."
21590,"[3282, 120, 596, 25, 263, 120, 596, 271, 1, 3,...",1,"[6.8177396e-06, 3.2601188e-06, 8.0305645e-06, ...",1,135434854,인형머리카락보다 더 가는 머리카락때문에 좋다는거 다 써봤는데 이건 처음부터 다르네 ...,"[인형, 머리, 카락, 더, 갈, 머리, 카락, 때문, 좋, 것, 다, 쓰, 보, ...","[인형, 머리, 카락, 더, 갈, 머리, 카락, 때문, 좋, 것, 다, 쓰, 보, ..."
6160,"[537, 77, 162, 55, 35, 906, 7, 3, 8, 88, 0, 0,...",1,"[7.1496474e-06, 5.125181e-06, 2.144847e-06, 2....",1,135087268,쫀쫀하고 부드럽고 올겨울 따뜻하게 치마입을것 같아요 감사합니다 ♡,"[쫀쫀, 부드럽, 올, 겨울, 따뜻, 치마, 입, 것, 같, 감사]","[쫀쫀, 부드럽, 올, 겨울, 따뜻, 치마, 입, 것, 같, 감사, 0, 0, 0,..."
16939,"[143, 4, 7, 45, 32, 35, 387, 684, 531, 92, 176...",1,"[7.0895094e-06, 6.2993013e-06, 1.0994604e-05, ...",1,134758977,기모가 있어 입었을때 편하고 따뜻했어요 다만 네벌 중 보라색은 외출복으론 색감이 좀...,"[기모, 있, 입, 때, 편하, 따뜻, 다만, 네, 벌, 중, 보라, 색, 외출, ...","[기모, 있, 입, 때, 편하, 따뜻, 다만, 네, 벌, 중, 보라, 색, 외출, ..."
14595,"[68, 275, 34, 5, 47, 30, 0, 0, 0, 0, 0, 0, 0, ...",1,"[1.697574e-06, 1.8634303e-06, 2.5636198e-06, 7...",1,134955763,다른 후기처럼 넘 잘 산듯요,"[다른, 후기, 넘, 잘, 살, 듯]","[다른, 후기, 넘, 잘, 살, 듯, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
2458,"[282, 91, 115, 75, 45, 752, 18, 43, 1489, 2919...",1,"[4.8662274e-05, 0.00011716495, 0.0002614107, 0...",1,134916382,견미리팩트 나올때부터 쭉 써왔다가 잠시 V 으로 눈을 돌렸는데 이번 시즌에 다시 돌...,"[견미리, 팩, 트, 나오, 때, 쭉, 쓰, 오, 잠시, v, 눈, 돌리, 이, 번...","[견미리, 팩, 트, 나오, 때, 쭉, 쓰, 오, 잠시, v, 눈, 돌리, 이, 번..."
31595,"[80, 144, 254, 1094, 23, 315, 1202, 63, 0, 0, ...",0,"[0.00013400958, 9.849804e-05, 7.666449e-05, 8....",1,134697130,아주 편합니다 면소재라서 트러블도 없고 몸에 감기는 느낌입니다,"[아주, 편, 소재, 트러블, 없, 몸, 감기, 느낌]","[아주, 편, 소재, 트러블, 없, 몸, 감기, 느낌, 0, 0, 0, 0, 0, ..."
9783,"[171, 210, 100, 133, 11, 92, 127, 26, 0, 0, 0,...",1,"[6.731922e-05, 5.2404466e-05, 3.114007e-05, 5....",1,134769177,몇 년전부터 계속 구매중입니다 늘 만족입니다,"[몇, 년, 전, 계속, 구매, 중, 늘, 만족]","[몇, 년, 전, 계속, 구매, 중, 늘, 만족, 0, 0, 0, 0, 0, 0, ..."


In [6]:
df_new = get_df(new_dt)
df_new.sample(10)

,x,y,alpha,pred,m_id,text,origin_tokens,tokens
10014,"[306, 17, 132, 3984, 163, 47, 55, 0, 0, 0, 0, ...",0,"[0.00060032454, 0.0013361599, 0.003552146, 0.0...",1,134913534,귀엽네요 사이즈는약간큰데 발도편하고 괜찮아요,"[귀엽, 사이즈, 약간, 큰데, 발, 편하, 괜찮]","[귀엽, 사이즈, 약간, 큰데, 발, 편하, 괜찮, 0, 0, 0, 0, 0, 0,..."
9298,"[216, 94, 1440, 6190, 1163, 37, 2757, 3, 122, ...",1,"[0.0025183887, 0.0016881408, 0.0023322885, 0.0...",1,134629465,오래 기다릴 줄 알았는데 다행히 한창 더울때 맞춰서 잘 왔구요 기사님이 너 무 친절...,"[오래, 기다릴, 알, 다행히, 한창, 덥, 때, 맞춰서, 잘, 왔, 기사, 님, ...","[오래, 알, 다행히, 한창, 덥, 때, 맞춰서, 잘, 왔, 기사, 님, 너, 친절..."
7083,"[1216, 122, 446, 1, 174, 534, 333, 315, 13, 19...",1,"[5.851646e-05, 0.00024374686, 3.63313e-05, 2.0...",1,134613301,드디어 왔네요 너무너무 좋아요 ㅎ 단품으로도 자주 구매했었는데 종에 이 가격이라니 ...,"[드디어, 왔, 너무너무, 좋, ㅎ, 단, 품, 자주, 구매, 13, 종, 이, 가...","[드디어, 왔, 너무너무, 좋, ㅎ, 단, 품, 자주, 구매, 13, 종, 이, 가..."
44835,"[1, 548, 278, 240, 31, 318, 529, 72, 1184, 831...",1,"[2.9762974e-05, 2.700018e-05, 1.4170557e-05, 1...",1,135791935,좋다는 샴푸 여러가지 써본중에 젤 부드럽게 와 닿는 제품입니다 저는 린스도 써지 않...,"[좋, 샴푸, 여러, 가지, 쓰, 본, 젤, 부드럽, 와, 닿, 제품, 제, 린스,...","[좋, 샴푸, 여러, 가지, 쓰, 본, 젤, 부드럽, 와, 닿, 제품, 제, 린스,..."
550,"[1, 133, 2734, 2580, 3, 10, 2, 0, 0, 0, 0, 0, ...",1,"[0.00013565896, 0.00012906326, 0.00019212386, ...",1,135149194,좋아요 D펜 잘쓰고 있어요,"[좋, 3, d, 펜, 잘, 쓸, 있]","[좋, 3, d, 펜, 잘, 쓸, 있, 0, 0, 0, 0, 0, 0, 0, 0, ..."
12874,"[2993, 12, 220, 7, 1893, 32, 57, 53, 8, 11, 17...",0,"[9.300427e-05, 7.1559676e-05, 0.0002388013, 0....",0,135447758,관찮기는 한데 너무 일찍 사서 다른 상품들 보니 살짝 드는 후회 정도,"[관, 않, 한데, 너무, 일찍, 살, 다른, 상품, 들, 보, 살짝, 들, 후회,...","[관, 않, 한데, 너무, 일찍, 살, 다른, 상품, 들, 보, 살짝, 들, 후회,..."
41764,"[2670, 107, 141, 63, 2388, 55, 0, 0, 0, 0, 0, ...",0,"[0.003236365, 0.007900761, 0.005426859, 0.0040...",1,135959613,라쿤털이 싸보여요 데님은 괜찮은듯,"[라쿤, 털, 싸, 보이, 데님, 괜찮]","[라쿤, 털, 싸, 보이, 데님, 괜찮, 0, 0, 0, 0, 0, 0, 0, 0,..."
14040,"[15, 27, 1, 24, 1633, 1378, 14, 31, 231, 1, 11...",1,"[0.0015654577, 0.0004723877, 0.00023378967, 0....",1,135445057,가격이 넘 좋아 만족함 유통기한 안에만 써도 가성비 좋음 발색도 좋고 구성도 좋음,"[가격, 넘, 좋, 만족, 유통, 기한, 안, 쓰, 가성비, 좋, 발색, 좋, 구성...","[가격, 넘, 좋, 만족, 유통, 기한, 안, 쓰, 가성비, 좋, 발색, 좋, 구성..."
42274,"[364, 7, 300, 3014, 32, 2562, 4278, 137, 0, 0,...",0,"[0.0007794013, 0.00019385233, 0.0012928246, 0....",1,135783664,설치 하기 너무 힘들었어요 ㅠ 드릴을 사다가 벽을 뚫음 ㅎ,"[설치, 너무, 힘들, 드릴, 살, 벽, 뚫, ㅎㅎ]","[설치, 너무, 힘들, 드릴, 살, 벽, 뚫, ㅎㅎ, 0, 0, 0, 0, 0, 0..."
5495,"[74, 13, 28, 8, 57, 51, 134, 311, 47, 617, 304...",1,"[3.5934776e-05, 1.2313037e-05, 3.8685157e-05, ...",1,135854173,하나 구매하고 맘에 들어서 다른 색상 재구매합니다 착용감 편하고 컵 위에 풀컵처럼 ...,"[하나, 구매, 맘, 들, 다른, 색상, 재구매, 착용감, 편하, 컵, 위, 풀, ...","[하나, 구매, 맘, 들, 다른, 색상, 재구매, 착용감, 편하, 컵, 위, 풀, ..."


In [7]:
# attention model 내에서 alpha값이 큰 순서대로 index 정렬
# "UNKNOWN" seq == 0 최초로 "UNKNOWN"이후 토큰들은 모두 "UNKNOWN"이므로 고려하지 않음
# (고려되는 토큰 길이 / 3) + 1개 고려
def get_max_args(row):
    alpha = np.array(row['alpha'])
    x =  np.array(row['x'])
    zero_index = np.where(x == 0)[0].tolist()
    if zero_index:
        sent_size = int(zero_index[0])
    else:
        sent_size = int(len(x))
    num_alpha = int(sent_size / 3) + 1
    alpha = alpha[:sent_size].argsort()[-num_alpha:][::-1]
    return alpha.tolist()

In [8]:
df_prev['max_args'] = df_prev.apply(lambda row: get_max_args(row), axis=1)
df_new['max_args'] = df_new.apply(lambda row: get_max_args(row), axis=1)

In [9]:
# 위에서 산출한 token idx를 token으로변환
def get_alpha_tokens(row):
    max_args = row['max_args']
    xs = row['x']
    tokens = row['tokens']
    alpha_tokens = list()
    try:
        alpha_tokens = [tokens[arg] for arg in max_args if xs[arg] != 0]
    except IndexError:
        print("IndexError")
        alpha_tokens = None
    finally:
        return alpha_tokens

In [10]:
df_prev['alpha_tokens'] = df_prev.apply(lambda row: get_alpha_tokens(row), axis=1)
df_new['alpha_tokens'] = df_new.apply(lambda row: get_alpha_tokens(row), axis=1)

In [11]:
df_new.loc[:, ['text', 'alpha_tokens', 'tokens', 'y', 'pred']].sample(10)

,text,alpha_tokens,tokens,y,pred
23349,분홍색을 좋아해서 집에서 입으려고 상품평도 안보고 싸서 얼른 구매했는데 촉감이 별로...,"[대비, 비싼, ㅎ, 가격, 그러, 뎅, 났, 안, 땀]","[분홍색, 색, 좋아해서, 집, 입, 상품평, 안, 보, 쌓, 얼른, 구매, 촉감,...",0,0
44706,두번째 구매에요 전 머리결을 일부러 부드럽게 하는 화학적성분이 없다고 하여 쟁여두고...,"[높, 만족, 알, 보, 사용, ㅎ, 하, 되, 구매, 또, 않]","[두, 번, 째, 구매, 전, 머리결, 일부러, 부드럽, 하, 화학, 적, 성분, ...",1,1
3537,브라캡 넣는 부분이 한 쪽이 바느질이 잘못돼서 너무 좁아서 칼로 재봉 부분 일부 절...,"[55, 니스, 복, 피트, 55, 44, 입고, 상의, 일반, 입, 할]","[브라, 캡, 넣, 부분, 한, 쪽, 바느질, 잘못, 되, 너무, 좁, 칼, 재봉,...",0,0
6812,스판성이 있어서 편하고 따뜻해서 겨울에 입기 좋아요 근데 팔이 조금 기네요,"[조금, 기, 팔, 근데, 좋]","[스판, 성, 있, 편하, 따뜻, 겨울, 입, 좋, 근데, 팔, 조금, 기, 0, ...",1,1
27664,중딩 딸아이 긴머리가 상해서 끈어지고 엉키고ㅠ 기존에 바르던 에센스와는 달리 뿌리는...,"[좋, 작, 용량, 좋, 함, 느낌, 풍성, 풀리, 잘, 킴, 후]","[중, 딩, 딸아이, 길, 머리, 상해서, 끌, 지, 엉키, 기존, 바르, 에센스,...",1,1
30739,너무 맘에 들어요 가격도 착하고 예쁜 디자인,"[가격, 맘, 착하]","[너무, 맘, 들, 가격, 착하, 예쁜, 디자인, 0, 0, 0, 0, 0, 0, ...",1,1
43464,잘받았어요 정사이즈 주문해야 하는데 한치수 올렸더니 좀 크네요 겨울이라 그냥 입으려...,"[입, 그냥, 잘, 입, 좀, 겨울]","[잘, 받, 정, 사이즈, 주문, 하, 한, 치수, 올리, 좀, 크, 겨울, 그냥,...",0,1
3793,배송빠르고 면좋아요 따뜻하겠어요 두꺼버요 조아요,"[배송, 따뜻, 빠르]","[배송, 빠르, 좋, 따뜻, 꺼, 벌, 조아, 0, 0, 0, 0, 0, 0, 0,...",1,1
5619,세면대에서 냄새가 스물스물 올라와서 구입했어요 사용하고나니 역겨운냄새는 쏙 빠지고 ...,"[한번, 빠지, 더, 잘, 물, 전, 구매]","[세면대, 냄새, 올라와서, 구입, 사용, 나, 역겹, 냄새, 쏙, 빠지, 물, 전...",1,1
42433,너무편하고 예뻐요 여태구매한것중 제일좋아요,"[좋, 여태, 제일]","[너무, 편하, 예쁘, 여태, 구매, 제일, 좋, 0, 0, 0, 0, 0, 0, ...",1,1


In [12]:
df_new.loc[(df_new.y == 0) & (df_new.pred == 0), ['text', 'alpha_tokens', 'tokens']].sample(10)

,text,alpha_tokens,tokens
36000,이음새 모양이 대칭이 아니고 다 엇갈림,"[대칭, 모양]","[이음새, 새, 모양, 대칭, 다, 0, 0, 0, 0, 0, 0, 0, 0, 0,..."
41915,오트밀샀었는데 너무 따뜻해서 재주문했어요 그런데 골지피치색은 골지가 삐뚤삐뚤 반품하...,"[봅니다, 그냥, 소비자, 자, 빼, 불량, 쌓, 신]","[오트밀, 샀었, 너무, 따뜻, 재주문, 그런데, 골지, 피하, 색, 골지, 반품,..."
42502,흡입구가 뒷면에만 있어 사용하기가 좀 불편하네요,"[불편, 좀, 면]","[구, 뒷면, 면, 있, 사용, 좀, 불편, 0, 0, 0, 0, 0, 0, 0, ..."
49830,어제힘들게 물건은받았지만 너무오래기다리고 두번에 배송지연과 연락두절등으로 그물건이 ...,"[합니까, 뭐, 안, vvip, 할까, 합니다, 싫, 당분간, gs]","[어제, 힘들, 물건, 받, 너무, 오래, 기다리, 두, 번, 배송, 지연, 연락,..."
14225,한치수 크게 주문했는데 불구하고 기장이 짧아요ㅡㅡ 인 제가 사이즈 입는데 을 구입했...,"[참, 하, 올라와, 종아리, 내리, 있, 밑단, 이, 일어나, 바지, 그리고]","[한, 치수, 크, 주문, 불구, 기장, 짧, 163, 인, 제, 55, 사이즈, ..."
49104,먼소리입니까 오늘아침만해도 목욜배송예저이라고뜨기만하고 연락한자없더니 배송완료라니요 ...,"[아침, 해도, 욜, 소리, 오늘, 목]","[멀, 소리, 오늘, 아침, 해도, 목, 욜, 배송, 뜨, 하, 연락, 없, 배송,..."
37869,생각보다 달력이 작네요 늦게배달이 되서 좀 그러네요,"[그러, 좀, 배달]","[생각, 달력, 작, 늦, 배달, 되, 좀, 그러, 0, 0, 0, 0, 0, 0,..."
42331,가지를 주문했는데 가지만보내셨네요 소스 국자가 빠졌이요 빠른 확인부탁드려요 오랫동...,"[오랫동안, 된, 일, 어찌, 부탁드리, 다니, 동안]","[4, 가지, 주문, 3, 가지, 보내, 소스, 빠지, 빠른, 확인, 부탁드리, 오..."
4007,주관적인 평이지만 원더브라 보다는 볼륨감 디자인이 떨어 지네요 줄 사람 있으면 다 ...,"[싶, 버리, 줘, 원더, 주관, 평]","[주관, 적, 평, 원더, 브라, 볼륨, 감, 디자인, 떨, 지, 줄, 사람, 있,..."
7770,글쎄요 어째 내복같은 느낌일까요 편하긴 하겠는데 너무 이지집안패션 같아서,"[같, 안, 패션, 집안, 하]","[글쎄, 어찌, 내복, 같, 느낌, 편하, 하, 너무, 집안, 안, 패션, 같, 0..."


In [13]:
df_new.loc[(df_new.y == 1) & (df_new.pred == 1), ['text', 'alpha_tokens', 'tokens']].sample(10)

,text,alpha_tokens,tokens
1265,이번주양말신고 등산가려구용,"[이번, 가리, 구용]","[이번, 번, 주, 양말, 신고, 등산, 가리, 구용, 0, 0, 0, 0, 0, ..."
39549,입부분이 좀 튀어 나와서 조금 우스광스러워요,"[좀, 튀, 조금]","[입, 부분, 좀, 튀, 나와서, 조금, 광, 스러워, 0, 0, 0, 0, 0, ..."
24338,발이편해요 일할때 자주신고있어요 강추입니다,"[발, 편해요, 일]","[발, 편해요, 일, 때, 자주, 신, 있, 강추, 0, 0, 0, 0, 0, 0,..."
23270,소재도 좋고 많이 무겁지 않고 색상도 차분한 카키네요,"[무겁, 많이, 차분]","[소재, 좋, 많이, 무겁, 않, 색상, 차분, 카키, 0, 0, 0, 0, 0, ..."
29766,부모님 선물로 사드렸어요 제가신어보니 편하고 좋더라구요,"[좋, 제, 신, 드리]","[부모님, 선물, 사, 드리, 제, 신, 보, 편하, 좋, 0, 0, 0, 0, 0..."
18531,답답한 거 싫어서 크게 주문했어요 넉넉하게 편하고 예뻐서 만족합니다,"[만족, 싫, 크]","[답답, 싫, 크, 주문, 넉넉, 편하, 예쁘, 만족, 0, 0, 0, 0, 0, ..."
11194,옷 정리 완전 편해졌네요 한눈에들어오고 꺼내입기 편해졌어요,"[정리, 완전, 옷, 꺼내, 편해졌]","[옷, 정리, 완전, 편해졌, 지, 한눈, 눈, 들어오, 꺼내, 입, 편해졌, 지,..."
7275,너무 예뻐요 무료배송도 감사 또 주문 할께요 번창하세요,"[번창, 무료, 배송, 감사]","[너무, 예쁘, 무료, 배송, 감사, 또, 주문, 할께요, 번창, 0, 0, 0, ..."
24123,미세먼지 심한 요즘 잘 사용하고있어요,"[심한, 먼지, 잘]","[미세먼지, 먼지, 심한, 요즘, 잘, 사용, 있, 0, 0, 0, 0, 0, 0,..."
3368,이것저것 다 써봐도 미샤금설이 최고인거 같아요 제 얼굴톤에도 가깝고 커버력도 좋고 ㄷ,"[ㄷ, 좋, 커버력, 얼굴, 제, 가깝]","[이것저것, 저것, 다, 쓰, 보, 미샤, 최고, 같, 제, 얼굴, 톤, 가깝, 커..."


In [14]:
df_prev['correct'] = (df_prev.y == df_prev.pred)
df_new['correct'] = (df_new.y == df_new.pred)

In [15]:
def get_accuarcy(df):
    return len(df.loc[(df.correct == True)]) / len(df)


def get_f1_score(df):
    tp = len(df.loc[(df.y == 1) & (df.pred == 1)])
    fp = len(df.loc[(df.y == 0) & (df.pred == 1)])
    tn = len(df.loc[(df.y == 1) & (df.pred == 0)])
    fn = len(df.loc[(df.y == 0) & (df.pred == 0)])
    
    pr_0 = fn / (fp+fn)
    rc_0 = fn / (tn+fn)
    f1_0 = 2*pr_0*rc_0/(pr_0+rc_0)

    pr_1 = tp / (tp+tn)
    rc_1 = tp / (tp+fp)
    f1_1 = 2*pr_1*rc_1/(pr_1+rc_1)

    return f1_0, f1_1

In [16]:
acc_prev = get_accuarcy(df_prev)
acc_new = get_accuarcy(df_new)
print("acc_prev : {}, acc_new: {}".format(acc_prev, acc_new))

acc_prev : 0.852729301948052, acc_new: 0.860129176980198


In [17]:
f1_neg_prev, f1_pos_prev = get_f1_score(df_prev)
f1_neg_new, f1_pos_new = get_f1_score(df_new)
print("f1_neg_prev : {}, f1_neg_new: {}".format(f1_neg_prev, f1_neg_new))
print("f1_pos_prev : {}, f1_pos_new: {}".format(f1_pos_prev, f1_pos_new))

f1_neg_prev : 0.6666666666666667, f1_neg_new: 0.6945136630485282
f1_pos_prev : 0.9054859189321179, f1_pos_new: 0.9093006633478375


In [18]:
alpha_tokens = df_new.loc[(df_new.correct == True) & (df_new.y == 1)].alpha_tokens.tolist()
counter = Counter(chain(*alpha_tokens))
alpha_count = sorted(counter.items(), key=operator.itemgetter(1))
alpha_count.reverse()